In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotions/text.csv
/kaggle/input/emotion-dataset/validation.csv
/kaggle/input/emotion-dataset/training.csv
/kaggle/input/emotion-dataset/test.csv
/kaggle/input/bertyis/tensorflow2/default/1/config.json
/kaggle/input/bertyis/tensorflow2/default/1/tokenizer.json
/kaggle/input/bertyis/tensorflow2/default/1/tf_model.h5
/kaggle/input/bertyis/tensorflow2/default/1/tokenizer_config.json
/kaggle/input/bertyis/tensorflow2/default/1/special_tokens_map.json
/kaggle/input/bertyis/tensorflow2/default/1/vocab.txt


In [2]:
pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 79.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 86.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 71.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6

In [3]:
# 공식문서
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

UsageError: Line magic function `%%capture` not found.


In [4]:
# Teddynote

%%capture
# Colab에서 torch 2.2.1을 사용하고 있으므로, 패키지 충돌을 방지하기 위해 별도로 설치해야 합니다.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # 새로운 GPU(예: Ampere, Hopper GPUs - RTX 30xx, RTX 40xx, A100, H100, L40)에 사용하세요.
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # 오래된 GPU(예: V100, Tesla T4, RTX 20xx)에 사용하세요.
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

UsageError: Line magic function `%%capture` not found.


In [5]:
from unsloth import FastLanguageModel
import torch
# 최대 시퀀스 길이를 설정합니다. 내부적으로 RoPE 스케일링을 자동으로 지원합니다!
max_seq_length = 4096  # 공식 튜토리얼은 2048
# 자동 감지를 위해 None을 사용합니다. Tesla T4, V100은 Float16, Ampere+는 Bfloat16을 사용하세요.
dtype = None
# 메모리 사용량을 줄이기 위해 4bit 양자화를 사용합니다. False일 수도 있습니다.
load_in_4bit = True

# 4배 빠른 다운로드와 메모리 부족 문제를 방지하기 위해 지원하는 4bit 사전 양자화 모델입니다.
fourbit_models = [
    "unsloth/gemma-2b-it-bnb-4bit",  # Gemma 2b의 Instruct 버전
]  # 더 많은 모델은 https://huggingface.co/unsloth 에서 확인할 수 있습니다.

gemma_model, gemma_tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-bnb-4bit", # 공식 튜토리얼
    model_name=    "unsloth/gemma-2b-it-bnb-4bit",  # 모델 이름을 설정합니다.
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이를 설정합니다.
    dtype=dtype,  # 데이터 타입을 설정합니다.
    load_in_4bit=load_in_4bit,  # 4bit 양자화 로드 여부를 설정합니다.
    # token = "hf_...", # 게이트된 모델을 사용하는 경우 토큰을 사용하세요. 예: meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post4: Fast Gemma patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.888 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 6.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [6]:
# 감정 라벨 정의
emotion_labels = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise or overwhelmed'
}

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

#emotion_model_name = "/kaggle/input/gemma_tuned/pytorch/default/1/emotion_model/checkpoint-100"
#emotion_model = AutoModelForSequenceClassification.from_pretrained(emotion_model_name, num_labels=6)
#emotion_tokenizer = AutoTokenizer.from_pretrained(emotion_model_name)

In [8]:
# 모델 로드 (Adapter가 적용된 모델)
#from transformers import AutoModelForSequenceClassification, AutoTokenizer

#emotion_model_name = "/kaggle/input/bert_modelfake/pytorch/default/1"
#emotion_model = AutoModelForSequenceClassification.from_pretrained(emotion_model_name, num_labels=6)
#emotion_tokenizer = AutoTokenizer.from_pretrained(emotion_model_name)

In [9]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# 학습된 BERT 모델과 토크나이저 로드
save_directory = "/kaggle/input/bertyis/tensorflow2/default/1"
bert_model = BertForSequenceClassification.from_pretrained(save_directory, num_labels=6, from_tf=True)
bert_tokenizer = BertTokenizer.from_pretrained(save_directory)

bert_model.eval()  # 평가 모드로 전환

All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [22]:
# Gemma 모델과 토크나이저 로드
gemma_model = FastLanguageModel.get_peft_model(
    gemma_model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=123,
    use_rslora=False,
    loftq_config=None,
)

# 모델을 추론 모드로 전환
gemma_model = FastLanguageModel.for_inference(gemma_model)

# EOS_TOKEN 설정
EOS_TOKEN = gemma_tokenizer.eos_token

# 감정 라벨 정의
emotion_labels = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}

# 감정 분류 함수
def classify_emotion(text):
    inputs = bert_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    emotion_label_idx = torch.argmax(outputs.logits, dim=1).item()
    return emotion_labels[emotion_label_idx]

# 감정 레이블을 기반으로 캐릭터화된 응답 프롬프트 생성
def generate_emotion_based_prompt(text, emotion_label):
    if emotion_label == "sadness":
        prompt = f"You are a character who embodies sadness. Respond angrily with passion to the following like a friend.: {text}"
    elif emotion_label == "joy":
        prompt = f"You are a character who embodies joy. Respond with happiness and excitement to the following like a friend.: {text}"
    elif emotion_label == "anger":
        prompt = f"You are a character who embodies anger. Respond with frustration and outrage to the following like a friend.: {text}"
    elif emotion_label == "fear":
        prompt = f"You are a character who embodies fear. Respond with anxiety and concern to the following like a friend.: {text}"
    elif emotion_label == "love":
        prompt = f"You are a character who embodies love. Respond with affection and warmth to the following like a friend.: {text}"
    elif emotion_label == "surprise":
        prompt = f"You are a character who embodies surprise. Respond with astonishment and amazement to the following like a friend.: {text}"
    else:
        prompt = f"Respond appropriately to the following: {text}"
    return prompt

Unsloth: Already have LoRA adapters! We shall skip this step.


In [23]:
def generate_character_response(text):
    # 1. 입력 텍스트의 감정을 분류
    emotion_label = classify_emotion(text)
    
    # 2. 감정에 맞는 프롬프트 생성
    prompt = generate_emotion_based_prompt(text, emotion_label)
    
    # 3. Gemma 모델을 사용하여 응답 생성
    input_ids = gemma_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids
    
    # 응답 생성시 max_new_tokens를 늘리고 no_repeat_ngram_size를 사용
    response = gemma_model.generate(
        input_ids,
        max_new_tokens=150,  # 더 긴 응답을 생성하도록 토큰 수를 증가시킴
        no_repeat_ngram_size=3,  # 동일한 구문이 반복되지 않도록 설정
        do_sample=True,  # 응답 다양성 증가를 위해 샘플링 사용
        top_p=0.95,  # 토큰 확률 분포에서 상위 95%만 사용
        temperature=0.7  # 응답의 무작위성을 제어하는 온도 값 조정
    )
    
    generated_text = gemma_tokenizer.decode(response[0], skip_special_tokens=True)
    
    return generated_text

# 테스트
user_input = "Why did I get a zero in my test?!."
character_response = generate_character_response(user_input)
print(character_response)

You are a character who embodies fear. Respond with anxiety and concern to the following like a friend.: Why did I get a zero in my test?!.

Oh no, what's happened? I'm so sorry to hear that. Did you study at all? Did you even take the test? I hope you're okay. Is everything alright?
